Imports

In [3]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
import pickle
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

Import dataset

In [4]:
with open('shape_features.csv', 'r') as data:
    df = pd.read_csv(data)

Shuffle data

In [5]:
from sklearn.utils import shuffle
data = shuffle(df)

Separate features from labels, and split into train and test sets. Drop useless features


In [6]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,0:-1]
y = df.iloc[:,-1]
columns = X.columns
print(columns)
X = X.drop(columns=['Max speed squared', 'Min speed squared', 'Total duration', 'Bounding box area', 'Movement', 'Bounding box height', 'Bounding box width', 'Length ratio'])

Index(['Angle of BB diagonal', 'Aspect', 'Average curvature',
       'Average squared speed', 'Bounding box area', 'Bounding box diagonal',
       'Bounding box height', 'Bounding box width', 'Convex hull area ratio',
       'Cos of first to last', 'Cos of initial angle', 'Curviness', 'DCR',
       'Density 1', 'Density 2', 'Distance first to last',
       'Enclosing shape5 ratio', 'Entropy', 'Least shape3 error',
       'Length perimeter ratio', 'Length ratio', 'Log BB area', 'Log aspect',
       'Log longest BB side', 'Log total length', 'Max curvature',
       'Max speed squared', 'Min speed squared', 'Movement', 'NDDE',
       'Openness', 'Overtracing', 'Perimeter efficiency', 'Perimeter to area',
       'Point ratio', 'Sharpness', 'Sin of first to last',
       'Sin of initial angle', 'Thinness ratio', 'Total angle',
       'Total angle / total length', 'Total duration', 'Total length',
       'Total length / BB diagonal', 'Width to height ratio'],
      dtype='object')


Feature Selection

In [7]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

selector = SelectFromModel(estimator=LogisticRegression(), threshold='1.5*mean').fit(X,y)
names = selector.get_feature_names_out(X.columns)
X = selector.transform(X)
X = pd.DataFrame(X, columns=names)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Dataset Statistics

In [8]:
print(str(len(X_train)) + " samples in the training set")
print(str(len(X_test)) + " samples in the test set")
print(str(len(X_train.columns)) + " features")
print(str(len(set(y_train))) + " labels")
print("Labels are:", set(y_train))
print("Features are:", set(X_train.columns))

189 samples in the training set
21 samples in the test set
6 features
5 labels
Labels are: {'shape4', 'shape1', 'shape2', 'shape5', 'shape3'}
Features are: {'Sharpness', 'Distance first to last', 'Bounding box diagonal', 'Density 1', 'Total length', 'Total angle'}


Train Model

In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(criterion='gini', verbose=False)
model.fit(X,y)

print("Training Accuracy", model.score(X_train, y_train)*100, "%")
print("------")
print("Test Accuracy", model.score(X_test, y_test)*100, "%")



Training Accuracy 100.0 %
------
Test Accuracy 100.0 %


Print Classification Report

In [14]:
from sklearn.metrics import classification_report
print(classification_report(model.predict(X_test),y_test))

              precision    recall  f1-score   support

      shape1       1.00      1.00      1.00         8
      shape2       1.00      1.00      1.00         3
      shape3       1.00      1.00      1.00         4
      shape4       1.00      1.00      1.00         2
      shape5       1.00      1.00      1.00         4

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21



Save Model

In [15]:
pickle.dump(model, open('Ex5_shape_classifier_random_forest.sav','wb'))